# 사전 설치

In [38]:
# !pip install wordcloud
# !pip install konlpy
# !pip install torch
# !sudo apt-get update
# !sudo apt-get install -y fonts-nanum fonts-noto-cjk
# !fc-list | grep -i "nanum\|noto"
# !sudo apt-get update
# !sudo apt-get install openjdk-11-jdk -y

# Library

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim

import math
import numpy as np
import pandas as pd

import re
import konlpy
from konlpy.tag import Okt
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from pathlib import Path

from typing import Optional, Dict

from sklearn.metrics import f1_score, accuracy_score, classification_report
from tqdm.auto import tqdm

import warnings
warnings.filterwarnings("ignore")
import pickle # Added for caching
import json # Added for vocab caching


# Data 준비
- 데이터를 로드하고, 추가 데이터셋을 병합

In [64]:
import pandas as pd
from typing import Tuple, Dict, List

def load_data(url: str = 'data/') -> pd.DataFrame:
    """원본 및 추가 데이터를 로드하고 병합하여 하나의 DataFrame으로 반환합니다."""
    print(f"Loading data from {url}...")

    # 1. 원본 train 데이터 로드 (idx 컬럼 제외)
    train = pd.read_csv(url + 'train.csv')

    # 2. 추가 일반 대화 데이터 로드 및 형식 맞추기 (예시)
    data_list = [train.drop(columns=['idx'], errors='ignore')]

    try:
        a = pd.read_csv(url + 'general_dialog1.csv').rename(columns={'dialogue': 'conversation'})
        a['class'] = '일반 대화'
        a = a[['class', 'conversation']]
        data_list.append(a)
    except FileNotFoundError:
        pass

    try:
        b = pd.read_csv(url + 'general_dialog2.csv').drop('Unnamed: 0', axis=1, errors='ignore')
        if 'class' not in b.columns: b['class'] = '일반 대화'
        data_list.append(b)
    except FileNotFoundError:
        pass

    # 3. 데이터 병합
    raw_data = pd.concat(data_list, axis=0, ignore_index=True)
    raw_data = raw_data.drop(columns=['Unnamed: 0'], errors='ignore')

    print(f"Total data loaded: {len(raw_data)} samples.")
    return raw_data

def create_label_maps(raw_data: pd.DataFrame) -> Tuple[Dict[str, int], Dict[int, str]]:
    """클래스 라벨을 ID로, ID를 라벨로 매핑하는 딕셔너리를 생성합니다."""
    # 일관된 순서를 위한 고정 라벨 사용
    all_labels = ["협박 대화", "갈취 대화", "직장 내 괴롭힘 대화", "기타 괴롭힘 대화", "일반 대화"]

    label2id = {label: i for i, label in enumerate(all_labels)}
    id2label = {v: k for k, v in label2id.items()}

    print(f"Labels mapped: {label2id}")
    return label2id, id2label

# Data Preprocessing
- 한국어 텍스트 전처리 및 형태소 분석 함수를 정의

In [65]:
import re
from konlpy.tag import Okt
from typing import List, Set, Optional

okt = Okt()
STOP_WORDS: Set[str] = {"하다"}

def preprocess_sentence(
    sentence: str,
    mode: str = "pos",
    stop_words: Optional[Set[str]] = None # Change type hint to Optional[Set[str]]
) -> List[str]:
    """문자열을 입력받아 정규화 및 형태소 분석을 수행하고 토큰 리스트를 반환합니다.

    mode:
        - "pos"    : 품사 기반 필터링 (명사, 동사, 형용사 등)
        - "nouns"  : 명사만 추출
        - "morphs" : 모든 형태소 추출
    """

    # 1. 정규화 및 클리닝
    sentence = str(sentence).strip()
    sentence = re.sub(r"[^가-힣0-9a-zA-Z.,!?~\s]", " ", sentence)  # 특수문자 제거
    sentence = re.sub(r"\n", " ", sentence)
    sentence = re.sub(r"([?.!,~])", r" \1 ", sentence)
    sentence = re.sub(r'\s{2,}', ' ', sentence).strip()

    if not sentence:
        return []

    # Ensure stop_words is an iterable set
    actual_stop_words = stop_words if stop_words is not None else set()

    # 2. 형태소 분석 모드별 처리
    try:
        if mode == "pos":
            include_tags = {"Noun", "Verb", "Adjective", "Exclamation", "Adverb"}
            pos_tags = okt.pos(sentence, stem=True, norm=True)
            tokens = [
                word for word, tag in pos_tags
                if tag in include_tags and len(word) > 1 and word not in actual_stop_words
            ]

        elif mode == "nouns":
            tokens = [word for word in okt.nouns(sentence)
                      if len(word) > 1 and word not in actual_stop_words]

        elif mode == "morphs":
            tokens = [word for word in okt.morphs(sentence, stem=True, norm=True)
                      if len(word) > 1 and word not in actual_stop_words]

        else:
            raise ValueError(f"지원하지 않는 mode: {mode}. ('pos', 'nouns', 'morphs' 중 선택)")

    except Exception:
        return []

    return tokens


# Vocab 빌드, 토큰 인코딩, PyTorch Dataset 및 DataLoader 구성

In [66]:
import json
import torch
import pandas as pd
from torch.utils.data import Dataset
from typing import List, Tuple, Dict, Iterable, Any
from collections import Counter

SPECIALS: List[str] = ["<pad>", "<unk>", "<cls>", "<sep>"]

class SimpleListDataset(Dataset):
    """PyTorch 학습/평가에 사용되는 기본 Dataset."""
    def __init__(self, records: List[Dict]):
        self.records = records
    def __len__(self):
        return len(self.records)
    def __getitem__(self, idx):
        return self.records[idx]

def build_vocab(
    token_lists: Iterable[List[str]],
    min_freq: int = 2,
    max_size: Optional[int] = 30000,
    specials: List[str] = SPECIALS,
) -> Tuple[Dict[str, int], List[str], Counter]:
    """토큰 리스트로부터 Vocab(stoi, itos)을 빌드합니다."""
    counter = Counter()
    for toks in token_lists:
        counter.update(toks)

    most = [tok for tok, cnt in counter.most_common() if cnt >= min_freq]
    if max_size is not None:
        cap = max_size - len(specials)
        most = most[:max(0, cap)]

    itos = list(specials) + most
    stoi = {tok: i for i, tok in enumerate(itos)}
    return stoi, itos, counter

def encode_tokens(
    tokens: List[str],
    stoi: Dict[str, int],
    max_len: int = 256,
    add_cls: bool = True,
    add_sep: bool = True,
) -> Tuple[List[int], List[int]]:
    """토큰 리스트를 input_ids와 attention_mask로 인코딩합니다."""
    pad_id = stoi["<pad>"]
    unk_id = stoi["<unk>"]
    cls_id = stoi.get("<cls>")
    sep_id = stoi.get("<sep>")

    ids = [stoi.get(t, unk_id) for t in tokens]

    extra = (1 if add_cls else 0) + (1 if add_sep else 0)
    keep = max_len - extra
    keep = max(0, keep)
    ids = ids[:keep]

    if add_cls and cls_id is not None:
        ids = [cls_id] + ids
    if add_sep and sep_id is not None:
        ids = ids + [sep_id]

    attn = [1] * len(ids)
    return ids, attn

def collate_batch(
    batch: List[Dict],
    pad_id: int,
) -> Dict[str, torch.Tensor]:
    """DataLoader를 위한 배치 패딩 및 텐서 변환 함수 (collate_fn)."""
    bs = len(batch)
    if bs == 0: return {}

    maxlen = max(len(x["input_ids"]) for x in batch)

    input_ids = torch.full((bs, maxlen), pad_id, dtype=torch.long)
    attention_mask = torch.zeros((bs, maxlen), dtype=torch.long)
    labels = torch.tensor([x["label"] for x in batch], dtype=torch.long)

    for i, x in enumerate(batch):
        L = len(x["input_ids"])
        input_ids[i, :L] = torch.tensor(x["input_ids"], dtype=torch.long)
        attention_mask[i, :L] = torch.tensor(x["attention_mask"], dtype=torch.long)

    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

def encode_data_frame(
    df: pd.DataFrame,
    stoi: Dict[str, int],
    label2id: Dict[str, int],
    max_len: int = 256
) -> List[Dict]:
    """DataFrame의 각 행을 인코딩 레코드 리스트로 변환합니다."""
    def encode_row(row):
        ids, attn = encode_tokens(row["tokens"], stoi, max_len=max_len, add_cls=True, add_sep=True)
        return {
            "input_ids": ids,
            "attention_mask": attn,
            "label": label2id.get(row["class"], -1),
        }

    df['tokens'] = df['tokens'].apply(lambda x: x if isinstance(x, list) else [])
    records = [encode_row(r) for _, r in df[df['tokens'].apply(len) > 0].iterrows()]
    return records

def save_vocab(path: str, itos: List[str]) -> None:
    """itos 리스트를 json 파일로 저장합니다."""
    with open(path, "w", encoding="utf-8") as f:
        json.dump(itos, f, ensure_ascii=False)

# Model

In [67]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from typing import Optional, Dict, List, Any

# ====================================================================
# I. Positional Encoding (Transformer용)
# ====================================================================
class PositionalEncoding(nn.Module):
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# ====================================================================
# Transformer 클래스 정의
# ====================================================================

class TransformerClassifier(nn.Module):
    """Transformer Encoder 기반 문서/대화 분류기"""
    def __init__(
        self,
        vocab_size: int,
        num_classes: int,
        emb_dim: int = 256,
        nhead: int = 8,
        num_layers: int = 4,
        dim_feedforward: int = 512,
        max_len: int = 256,
        dropout: float = 0.1,
        pad_id: int = 0,
    ):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_id)
        self.pos_encoder = PositionalEncoding(emb_dim, dropout, max_len)
        self.emb_dropout = nn.Dropout(dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=emb_dim,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers)

        # CLS 토큰을 이용한 풀링 (가장 앞 토큰)
        self.classifier = nn.Linear(emb_dim, num_classes)

        nn.init.xavier_uniform_(self.classifier.weight)
        nn.init.zeros_(self.classifier.bias)

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        return_repr: bool = False,
    ):
        # (B, S) -> (B, S, E)
        x = self.emb(input_ids) * math.sqrt(self.emb.embedding_dim) # Scaling (Optional but recommended)
        x = x.permute(1, 0, 2) # (S, B, E) for PositionalEncoding
        x = self.pos_encoder(x)
        x = self.emb_dropout(x)
        x = x.permute(1, 0, 2) # (B, S, E) for TransformerEncoder

        # Attention Mask (padding mask) 생성: 1=ignore, 0=not ignore
        src_key_padding_mask = (attention_mask == 0) if attention_mask is not None else None

        encoder_output = self.transformer_encoder(
            x,
            src_key_padding_mask=src_key_padding_mask
        )

        # CLS 토큰 풀링 (첫 번째 토큰 사용)
        sent_repr = encoder_output[:, 0, :]

        logits = self.classifier(sent_repr)

        if return_repr:
            return logits, sent_repr
        return logits

# ====================================================================
# LSTM 클래스 정의
# ====================================================================
class LSTMClassifier(nn.Module):
    """LSTM 기반 문서/대화 분류기"""
    def __init__(
        self,
        vocab_size: int,
        num_classes: int,
        emb_dim: int = 256,
        hidden_dim: int = 512,
        num_layers: int = 2,
        dropout: float = 0.1,
        pad_id: int = 0,
        bidirectional: bool = True,
    ):
        super().__init__()

        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_id)
        self.dropout_emb = nn.Dropout(dropout)
        self.bidirectional = bidirectional

        self.lstm = nn.LSTM(
            input_size=emb_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=bidirectional,
            batch_first=True
        )

        lstm_output_dim = hidden_dim * 2 if bidirectional else hidden_dim
        self.norm = nn.LayerNorm(lstm_output_dim)
        self.classifier = nn.Linear(lstm_output_dim, num_classes)

        nn.init.xavier_uniform_(self.classifier.weight)
        nn.init.zeros_(self.classifier.bias)

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        return_repr: bool = False,
    ):
        x = self.emb(input_ids)
        x = self.dropout_emb(x)

        if attention_mask is not None:
            lengths = attention_mask.sum(dim=1).cpu()
            x_packed = nn.utils.rnn.pack_padded_sequence(
                x, lengths, batch_first=True, enforce_sorted=False
            )
            # lstm_out, (h_n, c_n)
            _, (h_n, c_n) = self.lstm(x_packed)
        else:
            _, (h_n, c_n) = self.lstm(x)

        # h_n shape: (num_layers * num_directions, B, hidden_size)
        if self.bidirectional:
            # 마지막 레이어의 forward (h_n[-2])와 backward (h_n[-1])를 concat
            sent_repr = torch.cat([h_n[-2], h_n[-1]], dim=1)
        else:
            # 마지막 레이어의 hidden state
            sent_repr = h_n[-1]

        sent_repr = self.norm(sent_repr)
        logits = self.classifier(sent_repr)

        if return_repr:
            return logits, sent_repr
        return logits


# ====================================================================
# Cnn 클래스 정의
# ====================================================================
class CnnTextClassifier(nn.Module):
    """1D CNN (TextCNN) 기반 문서/대화 분류기"""
    def __init__(
        self,
        vocab_size: int,
        num_classes: int,
        emb_dim: int = 256,
        num_filters: int = 100,
        kernel_sizes: List[int] = [3, 4, 5],
        dropout: float = 0.5,
        pad_id: int = 0,
    ):
        super().__init__()

        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=pad_id)

        # 병렬 1D Conv 레이어
        self.convs = nn.ModuleList([
            nn.Conv1d(
                in_channels=emb_dim,
                out_channels=num_filters,
                kernel_size=k
            )
            for k in kernel_sizes
        ])

        self.classifier = nn.Linear(len(kernel_sizes) * num_filters, num_classes)
        self.dropout = nn.Dropout(dropout)

        nn.init.xavier_uniform_(self.classifier.weight)
        nn.init.zeros_(self.classifier.bias)

    def forward(
        self,
        input_ids: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None, # 호환성을 위해 받음
        return_repr: bool = False,
    ):
        # (B, S) -> (B, S, E)
        x = self.emb(input_ids)

        # (B, S, E) -> (B, E, S) [Conv1d 입력 형식: B, Channels, Length]
        x = x.permute(0, 2, 1)

        conv_outputs = []
        for conv in self.convs:
            conv_out = F.relu(conv(x)) # (B, num_filters, S_out)

            # Max-over-time pooling
            pooled_out = F.max_pool1d(conv_out, kernel_size=conv_out.shape[2]) # (B, num_filters, 1)
            conv_outputs.append(pooled_out.squeeze(2)) # (B, num_filters)

        # 모든 커널의 풀링 결과를 결합
        sent_repr = torch.cat(conv_outputs, dim=1)
        sent_repr = self.dropout(sent_repr)

        logits = self.classifier(sent_repr)

        if return_repr:
            return logits, sent_repr
        return logits



In [68]:
# ====================================================================
# III. 모델 생성 유틸리티 (config 기반 팩토리)
# ====================================================================
def create_model(config: Dict[str, Any], vocab_size: int, num_classes: int, pad_id: int) -> nn.Module:
    """
    config 딕셔너리를 기반으로 지정된 model_type의 모델 인스턴스를 생성하는 팩토리 함수.
    """
    model_type = config['model_type'].lower()

    # 1. 모든 모델에 공통적으로 필요한 파라미터 묶음
    common_params = {
        'vocab_size': vocab_size,
        'num_classes': num_classes,
        'emb_dim': config['emb_dim'],
        'dropout': config['dropout'],
        'pad_id': pad_id,
    }

    # 2. 모델 타입에 따라 필요한 파라미터만 추출하여 모델 생성
    if model_type == 'transformer':
        params = {
            'nhead': config['tf_nhead'],
            'num_layers': config['tf_num_layers'],
            'dim_feedforward': config['tf_dim_ff'],
            'max_len': config['max_len'],
        }
        return TransformerClassifier(**common_params, **params)

    elif model_type == 'lstm':
        params = {
            'hidden_dim': config['lstm_hidden_dim'],
            'num_layers': config['lstm_num_layers'],
            'bidirectional': config['lstm_bidirectional'],
        }
        return LSTMClassifier(**common_params, **params)

    elif model_type == 'cnn':
        params = {
            'num_filters': config['cnn_num_filters'],
            'kernel_sizes': config['cnn_kernel_sizes'],
        }
        return CnnTextClassifier(**common_params, **params)

    else:
        raise ValueError(f"Unsupported model_type: {model_type}. Must be 'transformer', 'lstm', or 'cnn'.")

# Train

In [69]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score, accuracy_score
from tqdm.auto import tqdm
from typing import Optional, Any, Tuple, List, Dict
from collections import Counter

# GPU 사용을 위해 AMP (Automatic Mixed Precision) 스케일러 정의
scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

def calculate_class_weights(labels: List[int], device: torch.device) -> torch.Tensor:
    """클래스 빈도 기반의 가중치를 계산합니다. (Inverse Frequency Weighting)"""
    counts = Counter(labels)
    num_classes = max(counts.keys()) + 1
    total_samples = len(labels)

    weights = [0.0] * num_classes
    for i in range(num_classes):
        # 1 / (클래스 빈도)
        weights[i] = total_samples / (counts.get(i, 1) * num_classes)

    weights_tensor = torch.tensor(weights, dtype=torch.float).to(device)
    # 클래스 가중치가 필요하지 않다면 (예: 불균형이 적다면) 아래 주석을 풀고 사용
    # return torch.ones(num_classes, dtype=torch.float).to(device)
    return weights_tensor

def train_one_epoch(
    model: nn.Module,
    loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: nn.Module,
    device: torch.device,
    scheduler: Optional[Any] = None,
) -> float:
    """한 에포크 동안 학습을 진행하고 평균 손실을 반환합니다."""
    model.train()
    total_loss = 0.0

    for batch in tqdm(loader, desc="Training"):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
            logits = model(input_ids, attention_mask)
            loss = criterion(logits, labels)

        # AMP를 사용한 역전파 및 최적화
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        total_loss += loss.item()

        if scheduler:
            scheduler.step()

    return total_loss / len(loader)

def evaluate(
    model: nn.Module,
    loader: DataLoader,
    criterion: nn.Module,
    device: torch.device
) -> Tuple[float, float, float, List[int], List[int]]: # Modified return type
    """검증 데이터셋으로 모델을 평가하고 손실, 정확도, F1 점수, 예측, 실제 라벨을 반환합니다."""
    model.eval()
    total_loss = 0.0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in tqdm(loader, desc="Validation"):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                logits = model(input_ids, attention_mask)
                loss = criterion(logits, labels)

            total_loss += loss.item()

            preds = torch.argmax(logits, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_loss = total_loss / len(loader)
    accuracy = accuracy_score(all_labels, all_preds)
    f1_macro = f1_score(all_labels, all_preds, average='macro', zero_division=0)

    return avg_loss, accuracy, f1_macro, all_preds, all_labels # Modified return values

def run_training_loop(
    model: nn.Module,
    train_loader: DataLoader,
    valid_loader: DataLoader,
    optimizer: torch.optim.Optimizer,
    device: torch.device,
    epochs: int,
    train_labels: List[int],
    scheduler: Optional[Any] = None,
    save_path: str = "./best_model.pt"
) -> Tuple[nn.Module, float]:
    """전체 학습 과정을 실행하고 최적 모델을 저장합니다."""

    # 클래스 가중치 계산 및 손실 함수 정의
    weights = calculate_class_weights(train_labels, device)
    criterion = nn.CrossEntropyLoss(weight=weights)

    best_f1 = -1.0
    best_epoch = 0

    for epoch in range(1, epochs + 1):
        print(f"\n--- Epoch {epoch}/{epochs} ---")

        train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device, scheduler)
        print(f"Train Loss: {train_loss:.4f}")

        valid_loss, valid_acc, valid_f1, _, _ = evaluate(model, valid_loader, criterion, device)
        print(f"Valid Loss: {valid_loss:.4f} | Acc: {valid_acc:.4f} | F1-Macro: {valid_f1:.4f}")

        # 최적 모델 저장 (F1-macro 기준)
        if valid_f1 > best_f1:
            best_f1 = valid_f1
            best_epoch = epoch
            torch.save(model.state_dict(), save_path)
            print(f"✨ Model saved at epoch {best_epoch} with F1-Macro: {best_f1:.4f}")

    print(f"\nTraining finished. Best F1-Macro: {best_f1:.4f} at epoch {best_epoch}")
    return model, best_f1

# 테스트 데이터 로드, 추론, 최종 submission 파일을 생성하는 함수

In [70]:
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from typing import Dict, Callable, List, Optional, Set, Tuple
from tqdm.auto import tqdm

class _TestListDataset(Dataset):
    """추론을 위한 테스트 데이터셋"""
    def __init__(self, records: List[Dict]):
        self.records = records
    def __len__(self):
        return len(self.records)
    def __getitem__(self, idx):
        return self.records[idx]

def _collate_fn_test(batch: List[Dict], pad_id: int) -> Dict[str, torch.Tensor]:
    """테스트 DataLoader를 위한 collate_fn (라벨 제외)"""
    bs = len(batch)
    if bs == 0: return {}

    maxlen = max(len(x["input_ids"]) for x in batch)

    input_ids = torch.full((bs, maxlen), pad_id, dtype=torch.long)
    attention_mask = torch.zeros((bs, maxlen), dtype=torch.long)

    for i, x in enumerate(batch):
        L = len(x["input_ids"])
        input_ids[i, :L] = torch.tensor(x["input_ids"], dtype=torch.long)
        attention_mask[i, :L] = torch.tensor(x["attention_mask"], dtype=torch.long)

    # idx도 함께 반환하여 submission 파일에 사용. 문자열 그대로 전달.
    idxs = [x["idx"] for x in batch]

    return {"input_ids": input_ids, "attention_mask": attention_mask, "idxs": idxs}

def get_predictions(
    model: nn.Module,
    test_loader: DataLoader,
    device: torch.device
) -> Tuple[List[int], List[str]]: # Changed return type for idxs to List[str]
    """테스트 로더를 통해 모델의 예측값과 원본 인덱스를 반환합니다."""
    model.eval()
    all_preds = []
    all_idxs = []

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Predicting"):
            # Debugging check: Ensure 'idxs' key exists
            if "idxs" not in batch:
                print(f"Error: Batch is missing 'idxs' key. Available keys: {batch.keys()}")
                raise KeyError("Batch from test_loader is missing 'idxs' key. Check _collate_fn_test.")

            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            idxs = batch["idxs"] # idxs는 이제 파이썬 리스트 (문자열) 입니다.

            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                logits = model(input_ids, attention_mask)

            preds = torch.argmax(logits, dim=1).cpu().numpy()
            all_preds.extend(preds)
            all_idxs.extend(idxs)

    return all_preds, all_idxs


def fill_submission_class(
    test_csv_path: str,
    submission_csv_path: str,
    model: nn.Module,
    stoi: Dict[str, int],
    encode_tokens: Callable,
    preprocess_fn: Callable,
    stop_words: Set[str],
    id2label: Dict[int, str],
    max_len: int = 256,
    output_label_as: str = "name",
    batch_size: int = 32,
    device: torch.device = torch.device("cpu"),
    mode: str = "pos", # Add mode parameter
    config: Dict[str, Any] = None # ADDED: Pass the full config object
) -> pd.DataFrame:
    """테스트 데이터를 로드, 전처리 및 추론 후 submission 파일 포맷으로 반환합니다."""

    # 1. 테스트 데이터 로드
    test_df = pd.read_csv(test_csv_path)
    if test_df.empty:
        raise ValueError(f"Test CSV at {test_csv_path} is empty. Cannot generate submission.")
    # test_df['idx'] = test_df['idx'].astype(int) # 'idx'는 문자열로 유지
    test_df['conversation'] = test_df['conversation'].astype(str).fillna("") # Ensure conversation is string

    # 2. 전처리 및 인코딩
    test_df['tokens'] = test_df['conversation'].apply(lambda s: preprocess_fn(s, mode=mode, stop_words=stop_words))

    pad_id = stoi["<pad>"]
    test_records = []

    # Determine required minimum sequence length for the current model
    required_min_len = 1 # Default minimum length
    if config and config['model_type'] == 'cnn':
        # Use DEFAULT_CONFIG if cnn_kernel_sizes not in config, though it should be if cnn_experiment_config is used
        current_kernel_sizes = config.get('cnn_kernel_sizes', [3,4,5]) # Fallback to default if not found
        required_min_len = max(current_kernel_sizes)

    for i, row in test_df.iterrows():
        ids, attn = encode_tokens(row["tokens"], stoi, max_len=max_len, add_cls=True, add_sep=True)
        # Filter out sequences that are too short for the current model architecture
        if len(ids) >= required_min_len:
            test_records.append({
                "idx": row["idx"], # idx는 문자열 그대로 저장
                "input_ids": ids,
                "attention_mask": attn,
            })

    if not test_records:
        raise ValueError("test_records is empty after preprocessing and encoding. Check your test data or preprocessing steps.")

    # 3. DataLoader 준비
    test_ds = _TestListDataset(test_records)
    collate_fn_test = lambda b: _collate_fn_test(b, pad_id)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False, collate_fn=collate_fn_test)

    # 4. 예측 수행
    preds_id, idxs = get_predictions(model, test_loader, device)

    # 5. 최종 submission 파일 포맷에 맞게 재정렬 및 라벨 변환
    submission_df = pd.read_csv(submission_csv_path)

    # 예측 결과를 idx에 맞게 매핑하여 class 컬럼을 업데이트
    pred_map = pd.Series(preds_id, index=idxs) # idxs는 이제 문자열 인덱스를 가집니다.
    submission_df['class'] = submission_df['idx'].map(pred_map)

    # 라벨 포맷 변환
    if output_label_as == "name":
        submission_df['class'] = submission_df['class'].map(id2label)

    # 예측하지 못한 값 (None/NaN)은 일반 대화 또는 0으로 처리 (데이터셋에 따라 다름)
    if output_label_as == "name":
         submission_df['class'] = submission_df['class'].fillna('일반 대화')
    else: # id인 경우
         submission_df['class'] = submission_df['class'].fillna(4).astype(int) # 4 = '일반 대화' ID

    return submission_df

# Main

In [71]:
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import os
from typing import List, Dict, Optional, Any, Tuple, Set
import copy
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# 파이썬화 할 경우
# # 모듈 import
# from data_load import load_data, create_label_maps
# from data_preprocessing import preprocess_sentence, STOP_WORDS
# from data_tokenization import (
#     build_vocab, encode_data_frame,
#     SimpleListDataset, collate_batch,
#     save_vocab, encode_tokens
# )
# from model import create_model
# from train import run_training_loop
# from submission import fill_submission_class


# ====================================================================
# I. Configuration
# ====================================================================
DEFAULT_CONFIG: Dict[str, Any] = {
    # --- General Config ---
    'data_url': 'data/',
    'test_csv_name': 'test.csv',
    'submission_csv_name': 'submission.csv',
    'vocab_path': 'vocab.json',
    'model_save_path': 'best_model.pt',
    'epochs': 10,
    'max_len': 256,
    'batch_size': 16,
    'lr': 2e-4,
    'dropout': 0.1,
    'seed': 42,

    # --- Model Selection ---
    'model_type': 'transformer', # 'transformer', 'lstm', 'cnn' 중 선택
    'emb_dim': 256,

    # --- Transformer Params (tf_ prefix) ---
    'tf_nhead': 8,
    'tf_num_layers': 3,
    'tf_dim_ff': 512,

    # --- LSTM Params (lstm_ prefix) ---
    'lstm_hidden_dim': 512,
    'lstm_num_layers': 2,
    'lstm_bidirectional': True,

    # --- CNN Params (cnn_ prefix) ---
    'cnn_num_filters': 100,
    'cnn_kernel_sizes': [3, 4, 5],
}

# ====================================================================
# II. Helper Functions (Pipeline Logic)
# ====================================================================
def _setup_seed(seed):
    """결과 재현을 위한 시드 설정"""
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def _print_config(config: Dict[str, Any], default_config: Dict[str, Any]):
    print(f"--- Experiment Configuration (Model: {config['model_type']}) ---")
    is_changed = False
    for k, v in config.items():
        if k not in default_config or v != default_config.get(k):
            print(f" * {k}: {v} (Default: {default_config.get(k)})")
            is_changed = True
    if not is_changed:
        print(" * (No parameters changed from default)")
    print("-" * 50)

def _prepare_data_and_vocab(
    config: Dict[str, Any],
    mode: str,
    stop_words: Set[str]
) -> Dict[str, Any]:
    """데이터 파이프라인 실행: 로드, 전처리, Vocab, DataLoader 생성. (캐싱 지원)"""
    print(f"--- 1. Data Preparation Start ---")

    # 캐시 파일 경로 정의
    vocab_path = config['vocab_path']  # 예: 'vocab.json'
    tokens_cache = f"tokens_cache_{mode}.pkl"  # 토큰화 결과 캐시
    data_cache = f"data_split_{mode}.pkl"      # train/valid split 캐시

    # 1. 캐시된 토큰 + vocab 있으면 바로 로드
    if os.path.exists(vocab_path) and os.path.exists(tokens_cache) and os.path.exists(data_cache):
        print(f"Loading cached vocab, tokens, and split from disk...")

        # Vocab 로드
        with open(vocab_path, 'r', encoding='utf-8') as f:
            itos = json.load(f)
        stoi = {tok: i for i, tok in enumerate(itos)}

        # 토큰화된 raw_data 로드
        with open(tokens_cache, 'rb') as f:
            raw_data = pickle.load(f)

        # train/valid split 로드
        with open(data_cache, 'rb') as f:
            split_data = pickle.load(f)
            train_df = split_data['train']
            valid_df = split_data['valid']
            label2id = split_data['label2id']
            id2label = split_data['id2label']

        print(f"Cache loaded! Vocab: {len(itos)}, Train: {len(train_df)}, Valid: {len(valid_df)}")

    else:
        # 2. 캐시 없으면 처음부터 처리
        print("No cache found. Processing from scratch...")
        raw_data = load_data(url=config['data_url'])
        label2id, id2label = create_label_maps(raw_data)

        print("Starting data preprocessing (tokenization)...")
        raw_data['conversation'] = raw_data['conversation'].astype(str).fillna("")
        raw_data['tokens'] = raw_data['conversation'].apply(
            lambda s: preprocess_sentence(s, mode=mode, stop_words=stop_words)
        )

        # 클래스 필터링
        class_counts = raw_data['class'].value_counts()
        problematic_classes = class_counts[class_counts < 2].index.tolist()
        if problematic_classes:
            warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')
            print(f"Warning: Removing classes with <2 samples: {problematic_classes}")
            raw_data = raw_data[~raw_data['class'].isin(problematic_classes)]

        # Vocab 빌드
        stoi, itos, _ = build_vocab(raw_data["tokens"], min_freq=1, max_size=20000)
        save_vocab(vocab_path, itos)  # 저장
        print(f"Vocab saved to {vocab_path}, size: {len(itos)}")

        # Train/Valid Split
        train_df, valid_df = train_test_split(
            raw_data, test_size=0.2, random_state=config['seed'], stratify=raw_data["class"]
        )

        # 캐시 저장
        with open(tokens_cache, 'wb') as f:
            pickle.dump(raw_data, f)
        with open(data_cache, 'wb') as f:
            pickle.dump({
                'train': train_df, 'valid': valid_df,
                'label2id': label2id, 'id2label': id2label
            }, f)
        print(f"Cache saved: {tokens_cache}, {data_cache}")

    # ------------------------------------------------------------------
    # 아래부터는 캐시 유무와 관계없이 동일하게 진행
    # ------------------------------------------------------------------
    pad_id = stoi["<pad>"]
    num_classes = len(label2id) # Ensure num_classes is defined in both cache and non-cache branches

    train_records = encode_data_frame(train_df, stoi, label2id, max_len=config['max_len'])
    valid_records = encode_data_frame(valid_df, stoi, label2id, max_len=config['max_len'])

    train_ds = SimpleListDataset(train_records)
    valid_ds = SimpleListDataset(valid_records)
    collate_fn = lambda b: collate_batch(b, pad_id)

    train_loader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True, collate_fn=collate_fn)
    valid_loader = DataLoader(valid_ds, batch_size=config['batch_size'] * 2, shuffle=False, collate_fn=collate_fn)

    print(f"DataLoader ready. Train: {len(train_records)}, Valid: {len(valid_records)}")

    return {
        'stoi': stoi, 'id2label': id2label, 'pad_id': pad_id, 'num_classes': num_classes,
        'train_loader': train_loader, 'valid_loader': valid_loader,
        'train_labels': [rec["label"] for rec in train_records],
        'vocab_size': len(itos),
    }

def _run_experiment(
    config: Dict[str, Any],
    data_artifacts: Dict[str, Any],
    device: torch.device,
    mode: str, # Add mode parameter
    stop_words: Set[str] # Add stop_words parameter
) -> Tuple[float, str]:
    """모델 학습 및 제출 파일 생성을 담당합니다."""
    print(f"\n--- 2. Training and Prediction Start ---")

    # 1. Model & Optimizer: config만 전달
    model = create_model(
        config=config,
        vocab_size=data_artifacts['vocab_size'],
        num_classes=data_artifacts['num_classes'],
        pad_id=data_artifacts['pad_id'],
    ).to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=1e-2)

    # 2. Train
    trained_model, best_f1 = run_training_loop(
        model=model,
        train_loader=data_artifacts['train_loader'],
        valid_loader=data_artifacts['valid_loader'],
        optimizer=optimizer,
        device=device,
        epochs=config['epochs'],
        train_labels=data_artifacts['train_labels'],
        save_path=config['model_save_path']
    )

    # 3. Submission: final_model 생성도 동일하게 config만 전달
    print("\n--- 3. Submission Generation ---")
    final_model = create_model(
        config=config,
        vocab_size=data_artifacts['vocab_size'],
        num_classes=data_artifacts['num_classes'],
        pad_id=data_artifacts['pad_id'],
    ).to(device)

    if os.path.exists(config['model_save_path']):
        final_model.load_state_dict(torch.load(config['model_save_path'], map_location=device))
    else:
        final_model.load_state_dict(trained_model.state_dict())
        print("Warning: Best model file not found, using the model after the last epoch.")

    # Recreate criterion for evaluation, ensuring consistent class weights
    weights = calculate_class_weights(data_artifacts['train_labels'], device)
    criterion_for_report = nn.CrossEntropyLoss(weight=weights)

    print("\n[Validation Report] Starting inference on Validation Data...")
    _, _, _, val_predictions, val_targets = evaluate(
        final_model,
        data_artifacts['valid_loader'],
        criterion_for_report,
        device
    )

    print("="*70)
    print(f"               Validation Set Classification Report ({config['model_type'].upper()} - Best Model)")
    print("="*70)

    # 클래스 이름 설정을 위해 num_classes를 사용합니다.
    num_classes = data_artifacts['num_classes']
    target_names = [f'Class {data_artifacts["id2label"][i]}' for i in range(num_classes)]

    print(classification_report(
        val_targets,
        val_predictions,
        target_names=target_names,
        digits=4
    ))
    print("="*70)

    submission_df = fill_submission_class(
        test_csv_path=config['data_url'] + config['test_csv_name'],
        submission_csv_path=config['data_url'] + config['submission_csv_name'],
        model=final_model,
        stoi=data_artifacts['stoi'],
        encode_tokens=encode_tokens,
        preprocess_fn=preprocess_sentence,
        stop_words=stop_words, # Pass stop_words parameter
        id2label=data_artifacts['id2label'],
        max_len=config['max_len'],
        batch_size=config['batch_size'] * 2, # 테스트 배치 사이즈
        device=device,
        mode=mode, # Pass mode parameter
        config=config # ADDED: Pass config here
    )

    final_submission_path = f"submission_{config['model_type']}_f1_{best_f1:.4f}.csv"
    submission_df.to_csv(final_submission_path, index=False)
    print(f"\nSubmission file saved to {final_submission_path}")

    return best_f1, final_submission_path

# ====================================================================
# III. Main Entry Point
# ====================================================================
def main(config_updates: Optional[Dict[str, Any]] = None,  mode: str = "pos",
    stop_words: Set[str] = None): # Add stop_words here to receive from call
    """실험을 설정하고 전체 파이프라인을 실행합니다."""
    # 1. Configuration Setup
    config = copy.deepcopy(DEFAULT_CONFIG)
    if config_updates:
        config.update(config_updates)

    _print_config(config, DEFAULT_CONFIG)

    # 2. Environment Setup
    _setup_seed(config['seed'])
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Use global STOP_WORDS if not provided
    if stop_words is None:
        stop_words = globals().get('STOP_WORDS', set())

    # 3. Data Pipeline 실행
    data_artifacts = _prepare_data_and_vocab(config, mode, stop_words)

    # 4. Experiment Pipeline 실행
    best_f1, final_submission_path = _run_experiment(config, data_artifacts, device, mode, stop_words)

    # 최종 결과 요약
    print("\n" + "=" * 50)
    print(f"🔥 Experiment Finished! Best Validation F1-macro: {best_f1:.4f}")
    print(f"📄 Submission saved at: {final_submission_path}")
    print("=" * 50)

In [74]:
if __name__ == '__main__':

    # --- 실험 설정 예시 ---
    stop_words = ["대리", "회사", "과장", "부장", "업무", "휴가", "퇴근", "자네"]
    mode = "pos"
    # 1. CNN 모델로 변경 및 하이퍼파라미터 조정
    print("1D - CNN")
    cnn_experiment_config = {
        'model_type': 'cnn',
        'epochs': 15,
        'lr': 5e-4,
        'emb_dim': 128,
        'batch_size': 32,
        'model_save_path': 'exp_cnn_15ep.pt',
        'cnn_num_filters': 64,
        'cnn_kernel_sizes': [2, 3, 4],
    }
    main(config_updates=cnn_experiment_config, mode=mode, stop_words=stop_words)

    # 2. LSTM 모델로 변경 및 레이어 수/히든 차원 조정
    print("LSTM")
    lstm_experiment_config = {
        'model_type': 'lstm',
        'epochs': 15,
        'lr': 1e-3,
        'dropout': 0.2,
        'model_save_path': 'exp_lstm_768_3l.pt',
        'lstm_hidden_dim': 768,
        'lstm_num_layers': 3,
        'lstm_bidirectional': False,
    }
    main(config_updates=lstm_experiment_config, mode=mode, stop_words=stop_words)

    # 3. 기본 Transformer 설정으로 실행
    print("Trasformer")
    transformer_experiment_config = {
        'model_type': 'transformer',
        'epochs': 15,       # Default
        'lr': 0.0002,       # Default
        'emb_dim': 256,     # Default
        'model_save_path': 'exp_transformer_def.pt', # 저장 경로 지정
        'tf_nhead': 8,      # Default Transformer parameter
        'tf_num_layers': 3, # Default Transformer parameter

    }

    main(config_updates=transformer_experiment_config, mode=mode, stop_words=stop_words)


1D - CNN
--- Experiment Configuration (Model: cnn) ---
 * model_save_path: exp_cnn_15ep.pt (Default: best_model.pt)
 * epochs: 15 (Default: 10)
 * batch_size: 32 (Default: 16)
 * lr: 0.0005 (Default: 0.0002)
 * model_type: cnn (Default: transformer)
 * emb_dim: 128 (Default: 256)
 * cnn_num_filters: 64 (Default: 100)
 * cnn_kernel_sizes: [2, 3, 4] (Default: [3, 4, 5])
--------------------------------------------------
Using device: cuda
--- 1. Data Preparation Start ---
No cache found. Processing from scratch...
Loading data from data/...
Total data loaded: 5049 samples.
Labels mapped: {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
Starting data preprocessing (tokenization)...
Vocab saved to vocab.json, size: 9556
Cache saved: tokens_cache_pos.pkl, data_split_pos.pkl
DataLoader ready. Train: 4038, Valid: 1010

--- 2. Training and Prediction Start ---

--- Epoch 1/15 ---


Training:   0%|          | 0/127 [00:00<?, ?it/s]

Train Loss: 1.1508


Validation:   0%|          | 0/16 [00:00<?, ?it/s]

Valid Loss: 0.8026 | Acc: 0.7188 | F1-Macro: 0.7154
✨ Model saved at epoch 1 with F1-Macro: 0.7154

--- Epoch 2/15 ---


...

Valid Loss: 0.7599 | Acc: 0.8198 | F1-Macro: 0.8161

Training finished. Best F1-Macro: 0.8276 at epoch 11

--- 3. Submission Generation ---

[Validation Report] Starting inference on Validation Data...


Validation:   0%|          | 0/32 [00:00<?, ?it/s]

               Validation Set Classification Report (TRANSFORMER - Best Model)
                   precision    recall  f1-score   support

      Class 협박 대화     0.7742    0.8045    0.7890       179
      Class 갈취 대화     0.7692    0.8163    0.7921       196
Class 직장 내 괴롭힘 대화     0.8531    0.7704    0.8097       196
  Class 기타 괴롭힘 대화     0.7814    0.7671    0.7742       219
      Class 일반 대화     0.9643    0.9818    0.9730       220

         accuracy                         0.8307      1010
        macro avg     0.8284    0.8280    0.8276      1010
     weighted avg     0.8315    0.8307    0.8305      1010



Predicting:   0%|          | 0/16 [00:00<?, ?it/s]


Submission file saved to submission_transformer_f1_0.8276.csv

🔥 Experiment Finished! Best Validation F1-macro: 0.8276
📄 Submission saved at: submission_transformer_f1_0.8276.csv


In [ ]:
if __name__ == '__main__':

    # --- 실험 설정 예시 ---
    stop_words = []
    mode = "pos"
    # 1. CNN 모델로 변경 및 하이퍼파라미터 조정
    print("1D - CNN")
    cnn_experiment_config = {
        'model_type': 'cnn',
        'epochs': 15,
        'lr': 5e-4,
        'emb_dim': 128,
        'batch_size': 32,
        'model_save_path': 'exp_cnn_15ep.pt',
        'cnn_num_filters': 64,
        'cnn_kernel_sizes': [2, 3, 4],
    }
    main(config_updates=cnn_experiment_config, mode=mode, stop_words=stop_words)

    # 2. LSTM 모델로 변경 및 레이어 수/히든 차원 조정
    print("LSTM")
    lstm_experiment_config = {
        'model_type': 'lstm',
        'epochs': 15,
        'lr': 1e-3,
        'dropout': 0.2,
        'model_save_path': 'exp_lstm_768_3l.pt',
        'lstm_hidden_dim': 768,
        'lstm_num_layers': 3,
        'lstm_bidirectional': False,
    }
    main(config_updates=lstm_experiment_config, mode=mode, stop_words=stop_words)

    # 3. 기본 Transformer 설정으로 실행
    print("Trasformer")
    transformer_experiment_config = {
        'model_type': 'transformer',
        'epochs': 15,       # Default
        'lr': 0.0002,       # Default
        'emb_dim': 256,     # Default
        'model_save_path': 'exp_transformer_def.pt', # 저장 경로 지정
        'tf_nhead': 8,      # Default Transformer parameter
        'tf_num_layers': 3, # Default Transformer parameter

    }

    main(config_updates=transformer_experiment_config, mode=mode, stop_words=stop_words)


# 지난 실험들

In [48]:
if __name__ == '__main__':

    # --- 실험 설정 예시 ---

    # 1. CNN 모델로 변경 및 하이퍼파라미터 조정
    print("1D - CNN")
    cnn_experiment_config = {
        'model_type': 'cnn',
        'epochs': 15,
        'lr': 5e-4,
        'emb_dim': 128,
        'model_save_path': 'exp_cnn_15ep.pt',
        'cnn_num_filters': 64,
        'cnn_kernel_sizes': [2, 3, 4],
    }
    main(config_updates=cnn_experiment_config, mode='nouns')

    # 2. LSTM 모델로 변경 및 레이어 수/히든 차원 조정
    print("LSTM")
    lstm_experiment_config = {
        'model_type': 'lstm',
        'epochs': 15,
        'lr': 5e-4,
        'dropout': 0.2,
        'model_save_path': 'exp_lstm_768_3l.pt',
        'lstm_hidden_dim': 768,
        'lstm_num_layers': 3,
        'lstm_bidirectional': False,
    }
    main(config_updates=lstm_experiment_config, mode='nouns')

    # 3. 기본 Transformer 설정으로 실행
    print("Trasformer")
    transformer_experiment_config = {
        'model_type': 'transformer',
        'epochs': 15,       # Default
        'lr': 5e-4,       # Default
        'emb_dim': 256,     # Default
        'model_save_path': 'exp_transformer_def.pt', # 저장 경로 지정
        'tf_nhead': 8,      # Default Transformer parameter
        'tf_num_layers': 3, # Default Transformer parameter

    }

    main(config_updates=transformer_experiment_config, mode='nouns')

1D - CNN
--- Experiment Configuration (Model: cnn) ---
 * model_save_path: exp_cnn_15ep.pt (Default: best_model.pt)
 * epochs: 15 (Default: 10)
 * lr: 0.0005 (Default: 0.0002)
 * model_type: cnn (Default: transformer)
 * emb_dim: 128 (Default: 256)
 * cnn_num_filters: 64 (Default: 100)
 * cnn_kernel_sizes: [2, 3, 4] (Default: [3, 4, 5])
--------------------------------------------------
Using device: cuda
--- 1. Data Preparation Start ---
Loading data from data/...
Total data loaded: 5049 samples.
Labels mapped: {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
Starting data preprocessing (tokenization)...
These samples will be excluded from train_test_split to enable stratification.
Data after filtering: 5048 samples.
Vocab size: 7871. Padding ID: 0
DataLoader ready. Train samples: 4038, Valid samples: 1009

--- 2. Training and Prediction Start ---

--- Epoch 1/15 ---


Training:   0%|          | 0/253 [00:00<?, ?it/s]

Train Loss: 1.0817


Validation:   0%|          | 0/32 [00:00<?, ?it/s]

Valid Loss: 0.7776 | Acc: 0.7106 | F1-Macro: 0.7059
✨ Model saved at epoch 1 with F1-Macro: 0.7059

--- Epoch 2/15 ---


...

Valid Loss: 0.7286 | Acc: 0.7939 | F1-Macro: 0.7885
✨ Model saved at epoch 15 with F1-Macro: 0.7885

Training finished. Best F1-Macro: 0.7885 at epoch 15

--- 3. Submission Generation ---

[Validation Report] Starting inference on Validation Data...


Validation:   0%|          | 0/32 [00:00<?, ?it/s]

               Validation Set Classification Report (TRANSFORMER - Best Model)
                   precision    recall  f1-score   support

      Class 협박 대화     0.7000    0.7430    0.7209       179
      Class 갈취 대화     0.6884    0.7551    0.7202       196
Class 직장 내 괴롭힘 대화     0.7919    0.8929    0.8393       196
  Class 기타 괴롭힘 대화     0.8323    0.6147    0.7071       218
      Class 일반 대화     0.9505    0.9591    0.9548       220

         accuracy                         0.7939      1009
        macro avg     0.7926    0.7929    0.7885      1009
     weighted avg     0.7988    0.7939    0.7918      1009



Predicting:   0%|          | 0/16 [00:00<?, ?it/s]


Submission file saved to submission_transformer_f1_0.7885.csv

🔥 Experiment Finished! Best Validation F1-macro: 0.7885
📄 Submission saved at: submission_transformer_f1_0.7885.csv


In [49]:
if __name__ == '__main__':

    # --- 실험 설정 예시 ---

    # 1. CNN 모델로 변경 및 하이퍼파라미터 조정
    print("1D - CNN")
    cnn_experiment_config = {
        'model_type': 'cnn',
        'epochs': 15,
        'lr': 5e-4,
        'emb_dim': 128,
        'batch_size': 32,
        'model_save_path': 'exp_cnn_15ep.pt',
        'cnn_num_filters': 64,
        'cnn_kernel_sizes': [2, 3, 4],
    }
    main(config_updates=cnn_experiment_config, mode='morphs')

    # 2. LSTM 모델로 변경 및 레이어 수/히든 차원 조정
    print("LSTM")
    lstm_experiment_config = {
        'model_type': 'lstm',
        'epochs': 15,
        'lr': 1e-3,
        'dropout': 0.2,
        'model_save_path': 'exp_lstm_768_3l.pt',
        'lstm_hidden_dim': 768,
        'lstm_num_layers': 3,
        'lstm_bidirectional': False,
    }
    main(config_updates=lstm_experiment_config, mode='morphs')

    # 3. 기본 Transformer 설정으로 실행
    print("Trasformer")
    transformer_experiment_config = {
        'model_type': 'transformer',
        'epochs': 15,       # Default
        'lr': 0.0002,       # Default
        'emb_dim': 256,     # Default
        'model_save_path': 'exp_transformer_def.pt', # 저장 경로 지정
        'tf_nhead': 8,      # Default Transformer parameter
        'tf_num_layers': 3, # Default Transformer parameter

    }

    main(config_updates=transformer_experiment_config, mode='morphs')

1D - CNN
--- Experiment Configuration (Model: cnn) ---
 * model_save_path: exp_cnn_15ep.pt (Default: best_model.pt)
 * epochs: 15 (Default: 10)
 * batch_size: 32 (Default: 16)
 * lr: 0.0005 (Default: 0.0002)
 * model_type: cnn (Default: transformer)
 * emb_dim: 128 (Default: 256)
 * cnn_num_filters: 64 (Default: 100)
 * cnn_kernel_sizes: [2, 3, 4] (Default: [3, 4, 5])
--------------------------------------------------
Using device: cuda
--- 1. Data Preparation Start ---
Loading data from data/...
Total data loaded: 5049 samples.
Labels mapped: {'협박 대화': 0, '갈취 대화': 1, '직장 내 괴롭힘 대화': 2, '기타 괴롭힘 대화': 3, '일반 대화': 4}
Starting data preprocessing (tokenization)...
These samples will be excluded from train_test_split to enable stratification.
Data after filtering: 5048 samples.
Vocab size: 10113. Padding ID: 0
DataLoader ready. Train samples: 4038, Valid samples: 1010

--- 2. Training and Prediction Start ---

--- Epoch 1/15 ---


Training:   0%|          | 0/127 [00:00<?, ?it/s]

Train Loss: 1.1328


Validation:   0%|          | 0/16 [00:00<?, ?it/s]

Valid Loss: 0.7421 | Acc: 0.7614 | F1-Macro: 0.7594
✨ Model saved at epoch 1 with F1-Macro: 0.7594

--- Epoch 2/15 ---


...

Valid Loss: 0.7664 | Acc: 0.8376 | F1-Macro: 0.8345

Training finished. Best F1-Macro: 0.8407 at epoch 7

--- 3. Submission Generation ---

[Validation Report] Starting inference on Validation Data...


Validation:   0%|          | 0/32 [00:00<?, ?it/s]

               Validation Set Classification Report (TRANSFORMER - Best Model)
                   precision    recall  f1-score   support

      Class 협박 대화     0.7421    0.7877    0.7642       179
      Class 갈취 대화     0.8773    0.7296    0.7967       196
Class 직장 내 괴롭힘 대화     0.8906    0.8724    0.8814       196
  Class 기타 괴롭힘 대화     0.7344    0.8584    0.7916       219
      Class 일반 대화     0.9952    0.9455    0.9697       220

         accuracy                         0.8426      1010
        macro avg     0.8479    0.8387    0.8407      1010
     weighted avg     0.8506    0.8426    0.8440      1010



Predicting:   0%|          | 0/16 [00:00<?, ?it/s]


Submission file saved to submission_transformer_f1_0.8407.csv

🔥 Experiment Finished! Best Validation F1-macro: 0.8407
📄 Submission saved at: submission_transformer_f1_0.8407.csv
